Create the paraphrase evaluation dataset to test each model

The dataset will combine different datasets from different sources
1. PAWS Dataset. [Link Here](https://huggingface.co/datasets/paws).

In [1]:
from datasets import load_dataset
import pandas as pd
import re

In [2]:
paws_data = load_dataset("paws", "labeled_final")
quora_data = load_dataset("quora")

Reusing dataset paws (/Users/sameermalik/.cache/huggingface/datasets/paws/labeled_final/1.1.0/8d567c6472623f42bd2cc635cad06932d0f0cd2f897db56013c1180f4317d338)
100%|██████████| 3/3 [00:00<00:00, 75.53it/s]
Using custom data configuration default
Reusing dataset quora (/Users/sameermalik/.cache/huggingface/datasets/quora/default/0.0.0/36ba4cd42107f051a158016f1bea6ae3f4685c5df843529108a54e42d86c1e04)
100%|██████████| 1/1 [00:00<00:00, 51.23it/s]


In [3]:
paws_df = pd.DataFrame(paws_data["test"])
quora_df = pd.DataFrame(quora_data["train"])

In [4]:
paws_df_filtered = paws_df[paws_df["label"]==1]
quora_df_filtered = quora_df[quora_df["is_duplicate"]==False]

In [5]:
def quora_to_sentence_pair(d):
       text = d["text"]
       return text[0], text[1]
quora_df_filtered["sentence1"], quora_df_filtered["sentence2"] = zip(*quora_df_filtered['questions'].map(quora_to_sentence_pair))

/var/folders/1d/d4mhd8d16rb8x5vglgfk9bv80000gn/T/ipykernel_80648/3728472927.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quora_df_filtered["sentence1"], quora_df_filtered["sentence2"] = zip(*quora_df_filtered['questions'].map(quora_to_sentence_pair))
/var/folders/1d/d4mhd8d16rb8x5vglgfk9bv80000gn/T/ipykernel_80648/3728472927.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quora_df_filtered["sentence1"], quora_df_filtered["sentence2"] = zip(*quora_df_filtered['questions'].map(quora_to_sentence_pai

We only need the input sentence

Data normalization:
- remove all sentences with special characters
- lowercase

In [21]:
l1 = list(paws_df_filtered["sentence1"])
l2 = list(quora_df_filtered["sentence1"])
eval_dataset = pd.DataFrame(l1+l2, columns=["sentence1"])

In [22]:
eval_dataset

,sentence1
0,"In January 2011 , the Deputy Secretary General..."
1,"During her sophomore , junior and senior summe..."
2,"His father emigrated to Missouri in 1868 , but..."
3,It is situated south of Köroğlu Mountains and ...
4,The Río Blanco mine is a large copper mine loc...
...,...
258558,What do you think of the removal of the MagSaf...
258559,How many keywords are there in the Racket prog...
258560,What is one coin?
258561,What is the approx annual cost of living while...


In [23]:
# the list of character we do not want to include
special_characters = '`[@_#$-+=%^&*()<>?/\|}{~:]\"'


In [24]:
eval_dataset["contains_special"] = eval_dataset["sentence1"].apply(lambda x: any(c in special_characters for c in x))

In [25]:
eval_dataset = eval_dataset[eval_dataset["contains_special"]==False]

In [26]:
# add a column for sentence length
eval_dataset["sentence_length"] = eval_dataset["sentence1"].apply(lambda x: len(x.split()))

/var/folders/1d/d4mhd8d16rb8x5vglgfk9bv80000gn/T/ipykernel_80648/2096775824.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_dataset["sentence_length"] = eval_dataset["sentence1"].apply(lambda x: len(x.split()))


In [27]:
eval_dataset

,sentence1,contains_special,sentence_length
1,"During her sophomore , junior and senior summe...",False,28
2,"His father emigrated to Missouri in 1868 , but...",False,27
3,It is situated south of Köroğlu Mountains and ...,False,14
4,The Río Blanco mine is a large copper mine loc...,False,19
5,The spectral levels of light that can be measu...,False,28
...,...,...,...
253345,The best fun programming talks,False,5
254079,My father has got brain stroke and I have lots...,False,33
254505,"How can I find a publisher, I know this is pos...",False,26
255062,What are some cheaper or better alternatives t...,False,13


In [28]:
eval_dataset = eval_dataset[(eval_dataset["sentence_length"]>=7) & (eval_dataset["sentence_length"]<=20)]

In [29]:
eval_dataset

,sentence1,contains_special,sentence_length
3,It is situated south of Köroğlu Mountains and ...,False,14
4,The Río Blanco mine is a large copper mine loc...,False,19
6,The Sunset Sunset Road comes from right and be...,False,13
7,Mr. Thuso Nokwanda Mbedu was born in Pietermar...,False,12
9,Owned by Rick and Sheri Dorritie is Megasaurus...,False,15
...,...,...,...
245846,"Why do the Indian media,Government takes the c...",False,17
246411,What percentage of people have GPS in use at a...,False,12
248660,if rating agencies downgrade a currency what i...,False,20
253098,What's the worst thing you been threw,False,7


In [30]:
from collections import Counter
Counter(eval_dataset["sentence_length"])

Counter({14: 135,
         19: 204,
         13: 134,
         12: 99,
         15: 152,
         17: 145,
         18: 177,
         16: 143,
         20: 148,
         11: 36,
         10: 37,
         9: 36,
         8: 24,
         7: 22})

In [31]:
from random import sample
# select random sentences with different lengths
sentences = []
for i in range(7, 21):
       # find list of sentences with length i
       temp_sentences = list(eval_dataset[(eval_dataset["sentence_length"]==i)]["sentence1"])
       if not temp_sentences:
              continue
       c = sample(temp_sentences, 10)
       sentences += c

len(sentences)


140

In [32]:
output_file = "eval.txt"
with open(output_file, 'w+') as fp:
       for item in sentences:
              # write each item on a new line
              fp.write("%s\n" % item)
